In [317]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import Accuracy


# 整理数据
# 将三个文件的数据整合到data中

# 导入三个数据集
happy = np.load(f'./process/happy.npy')[:8200]
normal = np.load(f'./process/normal.npy')[:8200]
sad = np.load(f'./process/sad.npy')[:8200]

In [318]:
# 预处理
happy = happy.reshape(-1, 1, 10)
normal = normal.reshape(-1, 1, 10)
sad = sad.reshape(-1, 1, 10)

happy_label = np.full((happy.shape[0],1),0)
normal_label = np.full((normal.shape[0],1),1)
sad_label = np.full((sad.shape[0],1),2)

In [321]:
# 预处理
happy = happy.reshape(-1, 82)
normal = normal.reshape(-1, 82)
sad = sad.reshape(-1, 82)

happy_label = np.full((happy.shape[0],1),0)
normal_label = np.full((normal.shape[0],1),1)
sad_label = np.full((sad.shape[0],1),2)

In [322]:
# 检查数据维度
print(happy.shape, normal.shape, sad.shape)
print(happy_label.shape,normal_label.shape,sad_label.shape)

# 训练集和测试集
# label: happy, normal, sad
train = np.stack([happy, normal, sad])
mean_train = np.mean(train, axis=1, keepdims=True)
label = np.stack([happy_label,normal_label,sad_label])
print('训练集：',train.shape,label.shape)
test = np.repeat(mean_train, train.shape[1], axis=1)
test_label = label[:,:,:]
# 使用均值填充测试集
print('测试集：',test.shape,test_label.shape)

(100, 82) (100, 82) (100, 82)
(100, 1) (100, 1) (100, 1)
训练集： (3, 100, 82) (3, 100, 1)
测试集： (3, 100, 82) (3, 100, 1)


In [407]:
# 定义模型
# 输入层
input = tf.keras.layers.Input((100,82))

# 卷积层1
conv1 = tf.keras.layers.Conv1D(32, 7, activation='relu')(input)
drop1 = tf.keras.layers.Dropout(0.2)(conv1)

conv2 = tf.keras.layers.Conv1D(64, 5, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.2)(conv2)

conv3 = tf.keras.layers.Conv1D(64, 5,  activation='relu',)(drop2)

max2p = tf.keras.layers.MaxPool1D(2)(conv3)

# 输出层
dense1 = tf.keras.layers.Dense(128,'relu')(max2p)
dense2 = tf.keras.layers.Dense(64, 'relu')(dense1)
output = tf.keras.layers.Dense(3, 'softmax')(dense2)

#
np.set_printoptions(suppress=True)

# 定义模型
model = tf.keras.Model(inputs=input,outputs=output)

adam = tf.keras.optimizers.Adam(0.0003)

model.compile(optimizer=adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model_195"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_249 (InputLayer)      [(None, 100, 82)]         0         
                                                                 
 conv1d_682 (Conv1D)         (None, 94, 32)            18400     
                                                                 
 dropout_582 (Dropout)       (None, 94, 32)            0         
                                                                 
 conv1d_683 (Conv1D)         (None, 90, 64)            10304     
                                                                 
 dropout_583 (Dropout)       (None, 90, 64)            0         
                                                                 
 conv1d_684 (Conv1D)         (None, 86, 64)            20544     
                                                                 
 max_pooling1d_104 (MaxPool  (None, 43, 64)            0 

In [402]:
model.fit(train,label,
          epochs=30,
          verbose=2)

z = model.predict(test)

Epoch 1/30


ValueError: in user code:

    File "e:\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "e:\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "e:\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "e:\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1131, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "e:\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1225, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "e:\anaconda3\Lib\site-packages\keras\src\engine\compile_utils.py", line 620, in update_state
        metric_obj.update_state(y_t, y_p, sample_weight=mask)
    File "

In [ ]:
z